In [223]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [225]:
df = pd.read_csv("battles_data.csv")

In [227]:
# title-Spalte splitten

# Alte Spalte umbenennen
df = df.rename(columns={'title': 'title_raw'})


# Verschiedene Pipe-Varianten durch ASCII-Pipe normalisieren
df['title_raw'] = df['title_raw'].str.replace(r'[⎪¦｜‖]', '|', regex=True)

# Splitten am ersten Vorkommen von '|' oder '//', nicht-einfangend
split_result = df['title_raw'].str.split(r'\s*(?:\||//)\s*', n=1, expand=True)

# Sicherstellen, dass immer zwei Spalten existieren
if split_result.shape[1] == 1:
    split_result[1] = ''

# Neue Spalten anlegen
df['title'] = split_result[0].str.strip()
df['info'] = split_result[1].str.strip()

In [229]:
# Prüfen: enthält 'vs' (case-insensitive)
df['has_vs'] = df['title_raw'].str.contains('vs', case=False, na=False)

df[['title_raw', 'has_vs']].head(10)

# Nur Zeilen behalten, die 'vs' enthalten
df = df[df['has_vs'] == True].reset_index(drop=True)

# Optional: die Hilfsspalte entfernen
df = df.drop(columns=['has_vs'])

In [231]:
# Contender trennen

# Regex: ersetze alle Varianten von vs/ vs. durch 'vs.'
df['title'] = df['title'].str.replace(r'\bvs\.?\b', 'vs.', case=False, regex=True)

# Split am ersten 'vs.'
split_result = df['title'].str.split(r'\s+vs\.\s+', n=1, expand=True)

# Neue Spalten
df['name1'] = split_result[0].str.strip()
df['name2'] = split_result[1].str.strip()

In [233]:
# Liga eintragen

df['league'] = np.where(df['info'].str.contains('DLTLLY', case=False, na=False) | 
                         df['title'].str.contains('DLTLLY', case=False, na=False), 'DLTLLY',
                  np.where(df['info'].str.contains('FOB', case=False, na=False) |
                           df['title'].str.contains('FOB', case=False, na=False), 'FOB',
                    np.where(df['info'].str.contains("Don't Flop", case=False, na=False) |
                           df['title'].str.contains("Don't Flop", case=False, na=False), "Don't Flop",
                        np.where(df['info'].str.contains('Breakthrough', case=False, na=False) |
                           df['title'].str.contains('Breakthrough', case=False, na=False), 'Breakthrough',''))))

# Zeile individuell anpassen
df.loc[587, ["league"]] = ["Don't Flop"]

# Zeilen anzeigen, bei denen 'special' leer ist
df_no_league = df[df['league'] == '']
df_no_league

,id,title_raw,upload_date,view_count,like_count,comment_count,url,status,title,info,name1,name2,league
1,SYo3zBQzwtQ,Aytee vs Karanova | 🏆 ON BEAT TITLE MATCH 2025...,20250316.0,208529,6257,755.0,https://www.youtube.com/watch?v=SYo3zBQzwtQ,active,Aytee vs. Karanova,"🏆 ON BEAT TITLE MATCH 2025 🏆| RapBattle, Berlin",Aytee,Karanova,
215,FgFwq0icwso,Kato vs Mr. Whyte // Fussball Battle // Panke ...,20200719.0,28097,730,175.0,https://www.youtube.com/watch?v=FgFwq0icwso,active,Kato vs. Mr. Whyte,Fussball Battle // Panke Garten Berlin // 2020,Kato,Mr. Whyte,
217,Tm9vJh_dk9k,Khacoby vs Notyzze // Corona Battle Package,20200628.0,27973,810,192.0,https://www.youtube.com/watch?v=Tm9vJh_dk9k,active,Khacoby vs. Notyzze,Corona Battle Package,Khacoby,Notyzze,
218,LOOhH4_OvNE,Mars B. vs BX // Corona Battle Package,20200626.0,11170,291,70.0,https://www.youtube.com/watch?v=LOOhH4_OvNE,active,Mars B. vs. BX,Corona Battle Package,Mars B.,BX,
219,bMXRfRmgPTA,Craze vs Burst // English 1 Rounder // Corona ...,20200623.0,6276,91,35.0,https://www.youtube.com/watch?v=bMXRfRmgPTA,active,Craze vs. Burst,English 1 Rounder // Corona Battle Package,Craze,Burst,
224,vb_w46X0rGE,Proton vs Craze // On Beat + Accapella // Coro...,20200607.0,21101,620,154.0,https://www.youtube.com/watch?v=vb_w46X0rGE,active,Proton vs. Craze,On Beat + Accapella // Corona Package,Proton,Craze,
432,KSx0djjt3nA,Mars B. TeaserRound vs. Merlin (POV),20170914.0,23131,449,69.0,https://www.youtube.com/watch?v=KSx0djjt3nA,active,Mars B. TeaserRound vs.. Merlin (POV),None,Mars B. TeaserRound vs.. Merlin (POV),None,


In [235]:
#alles, was keinen Eintrag in "league" hat, bekommt DLTLLY (geprüft!)

df.loc[df['league'] == '', 'league'] = "DLTLLY"

In [ ]:
# Namen in neue Spalte kopieren für Ansetzungsform > norm_name

df["norm_name #1"] = df["Name #1"]
df["norm_name #2"] = df["Name #2"]

In [ ]:
# Ansetzungsform bereinigen, Groß-Kleinschreibung

df["norm_name #1"] = df["norm_name #1"].str.title()
df["norm_name #2"] = df["norm_name #2"].str.title()
df["hidden"] = df["hidden"].str.title()

In [ ]:
# Ansetzungsform bereinigen, Klammern entfernen

df["norm_name #1"] = df["norm_name #1"].str.replace(r"\(.*\)", "", regex=True).str.strip()
df["norm_name #2"] = df["norm_name #2"].str.replace(r"\(.*\)", "", regex=True).str.strip()

In [ ]:
# einzelne Spalten individuell anpassen

df.loc[1037, ["Name #1", "hidden"]] = ["Ssynic", "BMCL Titelmatch"]
df.loc[1255, ["Name #1", "hidden"]] = ["Besser", "BMCL meets DLTLLY Rap Battle"]
df.loc[688, ["hidden"]] = ["Merlin abwesend, Craze kickt seine Parts alleine"]
df.loc[198, ["Name #2", "hidden"]] = ["Shizu", "Deelah abwesend, Shizu springt ein"]
df.loc[440, ["norm_name #2", "Location", "hidden"]] = ["Jollyjay", "Kontext", "Der Sir abwesend, JollyJay springt ein"]
df.loc[966, ["Name #1", "Name #2", "hidden"]] = ["Ekhead & CPE", "Brian Damage & Karma", "Sport Rap-Battle, DLTLLY & SPOX: Borussia Dortmund vs. FC Bayern München"]

In [ ]:
# Freestyle-Battles, Ankündigungen, Interviews, 2on2-Battles rausfiltern > df_battles

df_battles = df[~df["hidden"].str.contains('Interview|Ankündigung|Freestyle|Teaser|Provokation', na=False)]
df_battles = df_battles[~df_battles["Name #1"].str.contains(r'&|Freestyle|Ankündigung|PPV|\+', na=False)]
df_battles = df_battles[~df_battles["Name #2"].str.contains(r'&|Freestyle|Ankündigung|PPV|\+', na=False)]

# unten individuell ermittelte Dubletten entfernen
df_battles = df_battles.drop([228, 451])

In [ ]:
# Aliase auflisten für norm_name-Spalten

aliases = {
    "Triplebeat": "Triple",
    "Triple Beat": "Triple",
    "TRIPLE": "Triple",  
    
    ".Tecey": "Tecey",
    
    "Alice": "Atropatrap",
    "Atzenkalle -": "Atzenkalle",

    "Bahamakarma": "Karma",
    "B-Dad": "Bdad",
    "Andres Kaliba": "Bilal",
    "Bong Teggy - Rematch": "Bong Teggy",
    "Bronko Banano Aka Street": "Bronko Banano",
    "Street": "Bronko Banano",
    
    "Cashus K.": "Cashus K",
    "Capco Jonnes": "Capco Jones",
    "Capcojones": "Capco Jones",
    "Cemo": "Cemo Dafuq",
    "Change": "Change19",
    "Cleptomatic": "Clep",
    "Clepto": "Clep",
    "Craze' Parts": "Craze",

    "D.E.Z.": "D.E.Z",
    "Der Fischer -": "Der Fischer",
    "Desperados": "Despo",

    "Ein Letztes Mal: Yarambo": "Yarambo",

    "Falk": "Falki",
    "Fatcap -": "Fatcap",
    "Fate One": "Fate",
    "Fate1": "Fate",
    "Four Seven": "Fourseven",
    "Fraxone": "Frax One",

    "Gier -": "Gier",

    "Hansen - Rematch": "Hansen",
    "Herrn Sören": "Herr Sören",

    "Jack Dragon - Rematch!": "Jack Dragon",
    "Jean Gough Frais": "Jean-Gough Frais",
    "Jizi": "Ji-Zi",
    "Juse Ju -": "Juse Ju",

    "Khacoby": "Karkobi",
    "King Lil&#39;C'": "King Lil'C",
    "Kol&#39;ja": "Kol'ja",

    "Lavas": "Lavaş",

    "Main Moe -": "Main Moe",
    "Malik N.": "Malik",
    "Mars": "Mars B.",
    "Mighty P.": "Mighty P",
    "Mighty Mo": "Meidi",

    "Nichtdeintyp": "Nicht Dein Typ",

    "O&#39;Spella": "O'Spella",
    
    "P-Zak -": "P-Zak",
    "Pueblo Escobar": "Pablo",
    "Papi Schlauch -": "Papi Schlauch",
    "Phillie": "Phillie.45",
    "Presto -": "Presto",
    "Pretty Shitty Jiggy": "PrettyShittyJiggy",
    "Prettyshittyjiggy": "Pretty Shitty Jiggy",

    "Quasi": "Quasi Infamous",

    "Ryko J": "Ryko-J",

    "Schlawinor": "Schlawin0r",
    "Schlawin0R": "Schlawin0r",
    "Snakeheadrhyme": "SnakeHeadRhyme",

    "Tableddn Timmy": "TableddnTimmy",
    "Tableddntimmy": "TableddnTimmy",
    "Tierstar -": "Tierstar",
    "Tightammic -": "Tightammic",

    "Mr. Whyte": "Yaamann",
    "Yamann": "Yaamann",
    "Mr. White": "Yaamann",
    "Mr White": "Yaamann"
}

df_battles["norm_name #1"] = df_battles["norm_name #1"].replace(aliases)
df_battles["norm_name #2"] = df_battles["norm_name #2"].replace(aliases)

In [ ]:
# Battlerapper mit nur 2 Battles (beide Contender) rausfiltern > df_battles_clean

# Alle Namen aus beiden Spalten zählen
all_names = pd.concat([df_battles["norm_name #1"], df_battles["norm_name #2"]])
counts = all_names.value_counts()

# Bedingung: mindestens einer der beiden Namen kommt mehr als einmal vor
mask = df_battles.apply(lambda row: counts[row["norm_name #1"]] > 3 or counts[row["norm_name #2"]] > 3, axis=1)

# Gefiltertes DataFrame
df_battles_clean = df_battles[mask]

In [ ]:
# Liste der rausgeschmissenen nur-2-Battles
df_one_battle = df_battles[~mask]
df_one_battle

In [ ]:
# 1. Zeilen finden, die Klammern enthalten
mask_parens = df_battles_clean["Name #1"].str.contains(r"\(.*\)", na=False) | df_battles_clean["Name #2"].str.contains(r"\(.*\)", na=False)

# 2. Diese Zeilen in separatem DataFrame speichern
df_with_parens = df_battles_clean[mask_parens]


df_with_parens

In [ ]:
# Dubletten finden

df_battles_clean["pair_sorted"] = df.apply(
    lambda row: tuple(sorted([row["norm_name #1"], row["norm_name #2"]])),
    axis=1
)

# Schritt 2: alle Paare zählen
pair_counts = df_battles_clean["pair_sorted"].value_counts()

# Schritt 3: nur Paare mit mehr als 1 Auftreten auswählen
duplicate_pairs = pair_counts[pair_counts > 1].index.tolist()

# Schritt 4: Zeilen ausgeben, die zu den Duplikaten gehören
df_duplicates = df_battles_clean[df_battles_clean["pair_sorted"].isin(duplicate_pairs)]

df_duplicates

In [ ]:
# alle Namen, alphabetisch, zum Dubletten filtern

all_names = pd.concat([df_battles["norm_name #1"], df_battles["norm_name #2"]])
unique_names = sorted(all_names.unique())
unique_names